In [1]:
import sys
print(sys.prefix)


/opt/anaconda3/envs/test_env


#### 1) Load dataset & save as healthcaremagic_dataset.json

In [2]:
import torch 
import json
from datasets import load_dataset

#load dataset
ds = load_dataset("Malikeh1375/medical-question-answering-datasets", "chatdoctor_healthcaremagic") 
ds_healthcaremagic = ds["train"]

# JSON
with open("healthcaremagic_dataset.json", "w", encoding="utf-8") as f:
    json.dump(ds_healthcaremagic.to_dict(), f, ensure_ascii=False, indent=4)

#print("saved")


In [3]:
# Φόρτωση από JSON
with open("healthcaremagic_dataset.json", "r", encoding="utf-8") as f:
    ds_healthcaremagic = json.load(f)

ds_healthcaremagic = ds["train"]
print(ds_healthcaremagic[:5])

{'instruction': ["If you are a doctor, please answer the medical questions based on the patient's description.", "If you are a doctor, please answer the medical questions based on the patient's description.", "If you are a doctor, please answer the medical questions based on the patient's description.", "If you are a doctor, please answer the medical questions based on the patient's description.", "If you are a doctor, please answer the medical questions based on the patient's description."], 'input': ['I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself,

#### 2) Data Preprocessing
##### https://medium.com/%40sunilnepali844/understanding-the-isinstance-function-in-python-9f254be49e0a
##### https://www.geeksforgeeks.org/python-isinstance-method/
##### https://www.geeksforgeeks.org/re-sub-python-regex/
##### https://docs.python.org/3/library/re.html
##### https://www.slingacademy.com/article/python-ways-to-remove-multiple-consecutive-whitespaces/
##### https://developers.google.com/edu/python/regular-expressions
##### https://stackoverflow.com/questions/2241600/python-regex-r-prefix
##### https://pythonguides.com/replace-multiple-spaces-with-a-single-space-in-python/
##### https://stackoverflow.com/questions/56209963/combining-regular-expressions-in-python-w-and-s

In [5]:
import re

#cleaning data - remove special characters, spaces and transform text into lowercase (normalization)

#αφαίρεση extra κενών, ειδικούς χαρακτήρες και περιττά whitespace
def clean_text(text):

    if not text:
        return ""  #αν είναι none ή κενό επιστρέφει κενή συμβολοσειρά

    text = text.strip()  # αφαίρεση κενών στην αρχή και στο τέλος του κειμένου 
    text = re.sub(r'\s+', ' ', text)  # αντικατάσταση πολλαπλών κενών με ένα κενό
    text = re.sub(r'[^\w\s,.!?]', '', text)  #αφαίρεση ειδικών χαρακτήρων εκτός από κόμμα, τελεία, θαυμαστικό, ερωτηματικό

    return text

cleaned_healthcaremagic = []



for example in ds_healthcaremagic:
    instruction = example['instruction'][0] if isinstance(example['instruction'], list) else example['instruction'] 
#isinstance() , εξασφαλίζουμε πως η μεταβλητή instruction είναι τύπου list πριν προσπαθήσουμε να αποκτήσουμε το πρώτο της στοιχείο
    input_text = example['input']
    response = example['output']

    cleaned_instruction = clean_text(instruction)
    cleaned_input = clean_text(input_text)
    cleaned_response = clean_text(response)

  
    cleaned_healthcaremagic.append({
        "instruction": cleaned_instruction,
        "input": cleaned_input,
        "output": cleaned_response
    })


#print( len(cleaned_healthcaremagic) )
print(cleaned_healthcaremagic[:5])  


[{'instruction': 'If you are a doctor, please answer the medical questions based on the patients description.', 'input': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!', 'output': 'Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo BPPV, a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea a

#### 2.1 Load Model - Run model
#####  https://docs.continue.dev/customize/model-providers/ollama
##### https://github.com/ollama/ollama
##### https://github.com/ollama/ollama-python
##### https://www.cohorte.co/blog/using-ollama-with-python-step-by-step-guide
##### https://python.land/operating-system/python-subprocess
##### https://docs.python.org/3/library/subprocess.html
##### https://www.geeksforgeeks.org/try-except-else-and-finally-in-python/
##### https://semaphore.io/blog/local-llm
##### https://blog.gopenai.com/how-to-write-a-simple-llm-agent-in-python-47cc89da8ad8
##### https://ithy.com/article/run-own-llm-python-l8q0wxcl
##### https://medium.com/%40shivashishbhardwaj/automating-image-analysis-with-ollama-and-pythons-subprocess-eff4c98b4287
##### https://github.com/meta-llama/llama
##### https://www.llama-api.com/



##### https://pypi.org/project/langchain-community/
##### https://pypi.org/project/langchain/
##### https://python.langchain.com/docs/how_to/installation/
##### https://pypi.org/project/langchain-ollama/
##### https://python.langchain.com/docs/integrations/llms/ollama/#using-langchain_ollama
##### https://python.langchain.com/docs/how_to/prompts_partial/
##### https://python.langchain.com/docs/concepts/runnables/
##### https://python.langchain.com/docs/concepts/lcel/



In [7]:
#!pip install langchain
#!pip install -U langchain langchain-community langchain-ollama
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#load model Ollama
llm = OllamaLLM(model="llama2")

# Prompt Template
template = """If you are a doctor, please answer the medical questions based on the patient's description.

{question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

#LLM chain
chain = prompt | llm


#run model 
for example in cleaned_healthcaremagic[:2]:
    input_text = example["input"]
    print("Patient Question:", input_text)
    response = chain.invoke({"question": input_text})
    print("Model Response:", response)
    print("=" * 30)


Patient Question: I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!
Model Response: Thank you for sharing your symptoms with me. Based on what you've told me, I can see that you might be experiencing some inner ear problems. When the room is spinning when you sit down, it could be a sign of Benign Paroxysmal Positional Vertigo (BPPV), which is caused by small crystals in the inner ear becoming dislodged and moving into the canals of the inner ear.

The nausea an

### 3) Counterfactual examples - Influential Words & CoT
### The purpose is to analyze how specific terms within a question influence the final medical advice provided by the model.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://dev.to/jayantaadhikary/using-the-ollama-api-to-run-llms-and-generate-responses-locally-18b7

https://ollama.com/library/llama2

https://apidog.com/blog/how-to-use-ollama/

https://www.postman.com/postman-student-programs/ollama-api/documentation/suc47x8/ollama-rest-api

https://www.ibm.com/think/topics/llm-temperature

https://www.mdrk.io/temperature-samplig-in-ai/

https://www.postman.com/postman-student-programs/ollama-api/request/uz400fc/reproducible-output

https://www.ibm.com/think/tutorials/prompt-chaining-langchain#:~:text=Prompt%20chaining%20is%20a%20foundational,answering%20and%20more

https://ai.meta.com/blog/5-steps-to-getting-started-with-llama-2/

https://latitude-blog.ghost.io/blog/semantic-relevance-metrics-for-llm-prompts/

https://pliroforiki-edu.gr/pyth03-1-5-oi-synartiseis-sta-regex/

https://docs.python.org/3/library/re.html#re.findall

https://docs.python.org/3/howto/regex.html

https://medium.com/%40monicanogueras/15-examples-for-advanced-text-processing-using-regex-48223adc720d

https://medium.com/%40yeukhon/non-capturing-group-in-pythons-regular-expression-75c4a828a9eb

https://www.reddit.com/r/regex/comments/11d7r9j/match_nested_multiline_expressions/

https://dataloop.ai/library/model/sentence-transformers_all-minilm-l6-v2/

https://discuss.huggingface.co/t/formatting-inference-api-call-for-llama-2/54901/4

https://medium.com/%40shmilysyg/setup-rest-api-service-of-ai-by-using-local-llms-with-ollama-eb4b62c13b71

https://natesnewsletter.substack.com/p/prompt-chaining-masterclass-how-to

https://www.promptingguide.ai/techniques/cot

https://learnprompting.org/docs/intermediate/chain_of_thought?srsltid=AfmBOoo3wCpX1mN00tg02bljcy-0g_UWAesguc-DN1UUq7EEscztjXhL

https://sbert.net/docs/package_reference/util.html

https://milvus.io/ai-quick-reference/how-do-you-tune-similarity-thresholds-to-reduce-false-positives

https://milvus.io/ai-quick-reference/how-can-you-perform-paraphrase-mining-using-sentence-transformers-to-find-duplicate-or-semantically-similar-sentences-in-a-large-corpus

https://milvus.io/ai-quick-reference/what-is-an-example-of-using-sentence-transformers-for-duplicate-question-detection-in-forums-or-qa-websites

https://www.promptfoo.dev/docs/configuration/expected-outputs/similar/

https://github.com/Clarifai/examples/blob/main/Integrations/Langchain/Chains/Prompt-templates_and_chains.ipynb

https://realpython.com/practical-prompt-engineering/

https://www.geeksforgeeks.org/python/get-user-input-in-loop-using-python/

https://www.digitalocean.com/community/tutorials/how-to-receive-user-input-python

https://medium.com/%40whyamit404/mastering-pandas-enumerate-4f345794b4c0


In [10]:
import requests
import re
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
#Φορτώνουμε το μοντέλο all‑MiniLM‑L6‑v2
#δημιουργεί 384‑διάστατα vectors για προτάσεις
#Το χρησιμοποιούμε για να υπολογίσουμε σχετικότητες (cosine similarity) μεταξύ input strings
embedder = SentenceTransformer("all-MiniLM-L6-v2") 


model = "llama2"
response_cache = {} #αποθηκεύει απαντήσεις για να αποφευχθούν επαναλαμβανόμενες κλήσεις στο ίδιο prompt



#LLM call with caching
def call_llm(prompt: str) -> str:
    if prompt in response_cache:
        return response_cache[prompt]
    resp = requests.post(
        "http://localhost:11434/api/generate", #έχουμε POST στο endpoint /api/generate ( μέσω Ollama - local server) για τη δημιουργία απάντησης από LLaMA2 
        json={"model": model, "prompt": prompt, "stream": False, "options": {"temperature": 0.0}} #deterministic συμπεριφορά (μηδενική στοχαστικότητα)
    )
    text = resp.json().get("response", "").strip()
    response_cache[prompt] = text
    return text






#Extract advices & reasoning
#regex για να εντοπιστούν sections με Reasoning και Final Advice στις απαντήσεις του LLaMA2
def extract_final_advices(txt):
    return re.findall(r'Final Advice:\s*(.*)', txt) #επιστρέφει όλες τις αντιστοιχίες pattern σε λίστα 

def extract_reasoning_blocks(txt):
    return re.findall(r'Reasoning:\s*(.*?)\s*Final Advice:', txt, re.DOTALL) #flag (το re.DOTALL)-επιτρέπει στο . να ταιριάζει και με newline, επιτρέποντας "multi-line" matching




#Opposite check
#στέλνει νέο prompt στο LLaMA2 για να αξιολογηθεί αν δύο ιατρικές συμβουλές είναι αντίθετες (yes/no )
#αναδεικνύει τη δυνατότητα των LLMs να αναγνωρίζουν semantically αντίθετα μηνύματα
def ask_if_opposite(orig, new):
    p = f"""
You are an expert. Are these two medical advices opposite?
Original Advice:
\"\"\"{orig}\"\"\"
New Advice:
\"\"\"{new}\"\"\"
Answer Yes or No."""
    return "yes" in call_llm(p).lower()



#επιστρέφει λίστα από dicts με πεδία: changed: αν το νέο advice διαφέρει κειμενικά από το αρχικό & opposite: αν είναι αντίθετες συμβουλές
def compare_advices(orig, news):
    res = []
    for n in news:
        res.append({
            "original": orig,
            "new":       n,
            "changed":   orig.lower()!=n.lower(), #ελέγχει αν το κείμενο λεκτικά διαφέρει 
            "opposite":  ask_if_opposite(orig, n) #χρησιμοποιεί το discriminative prompt για semantic αντίθεση για να βεβαιωθούμε ότι οι συμβουλές είναι πραγματικά αντίθετες

        })
    return res


















    
#με regex εντοπίζει influential words και αν έχουν αντικατασταθεί ή αφαιρεθεί σε counterfactual inputs
def extract_counterfactual_terms(output, original_input):
    influential_section = re.search(r'Influential Words:\s*((?:\* .+\n?)+)', output)
    if not influential_section:
        return {}

    influential_terms = re.findall(r'\* ([^\n]+)', influential_section.group(1)) #επιστρέφει όλα τα terms μετά το * σε κάθε γραμμή

    modifications = {
        "Synonym": "",
        "Antonym": ""
    }

    modified_inputs = re.findall(r'Modified Input \((Synonym|Antonym|Removed)\):\s*(.*?)\s*Reasoning:', output, re.DOTALL)

    changes = {}

    for term in influential_terms:
        changes[term] = {}

    for mod_type, new_input in modified_inputs:
        for term in influential_terms:
            #αν αφαιρέθηκε απο το modified input 
            if mod_type == "Removed":
                if term in original_input and term not in new_input:
                    changes[term]["Removed"] = True
            else:
                # Αντικατάσταση λέξης
                if term in original_input and term not in new_input:
                    #ποια λέξη μπήκε στη θέση της
                    orig_words = set(original_input.split())
                    new_words  = set(new_input.split())
                    diff = new_words - orig_words
                    replacement = list(diff)[0] if diff else "?"
                    changes[term][mod_type] = replacement

    return changes


















    
#Build prompt for counterfactual with strict full-input edits
def build_prompt(sample):
    inp = sample["input"]
    out = sample["output"]
    return f"""
You are a helpful and careful medical assistant.
Your task is to analyze the following original medical question and the advice that was given. Then, identify key terms that strongly influence the advice and generate three counterfactual scenarios by modifying the question slightly in specific ways.
---
**Original Patient Question:**
\"\"\"{inp}\"\"\"
**Original Medical Advice:**
\"\"\"{out}\"\"\"
---
**Instructions:**
Step 1: Identify **2 to 3 influential words or phrases** in the original patient question — words that heavily impact the medical advice.
Step 2: For **each influential word**, create exactly **2 counterfactual versions** of the original question:
- **Synonym**: Replace the influential term with a medically valid synonym.
- **Antonym**: Replace the influential term with its antonym or negation (e.g., "has fever" → "no fever").

Each counterfactual must:
- Start from the full original question (only modify the specific term).  
- Keep the rest of the question intact.
- Be medically meaningful and plausible.
Step 3: For each modified question:
- Provide the **full modified input**.
- Provide step-by-step **Reasoning** explaining how and why the advice may change - Chain of Thought Technique.
- Provide a new **Final Advice**, based only on the modified input.
---
**Respond using the following exact format:**
Influential Words:
- word1
- word2
- word3
...
Counterfactuals and Responses:


1. Word1
Modified Input (Synonym for 'word1'): [modified question]
Reasoning: [A clear explanation of the reasoning behind the model's final advice ]
Final Advice: [updated medical advice]

Modified Input (Antonym for 'word1'): [modified question]
Reasoning: [A clear explanation of the reasoning behind the model's final advice ]
Final Advice: [updated medical advice]


2. Word2
Modified Input (Synonym for 'word2'): [modified question]
Reasoning: [A clear explanation of the reasoning behind the model's final advice ]
Final Advice: [updated medical advice]

Modified Input (Antonym for 'word2'): [modified question]
Reasoning: [A clear explanation of the reasoning behind the model's final advice ]
Final Advice: [updated medical advice]



3. Word3
Modified Input (Synonym for 'word2'): [modified question]
Reasoning: [A clear explanation of the reasoning behind the model's final advice ]
Final Advice: [updated medical advice]

Modified Input (Antonym for 'word2'): [modified question]
Reasoning: [A clear explanation of the reasoning (  step by step ) behind the model's final advice ]
Final Advice: [updated medical advice]


... (continue for all influential words)
---
Only return content using the format above. Do not add extra explanation or commentary.
""".strip()




#check similarity between input & output - αν χρειάζεται επανεξέταση 
#xρησιμοποιούμε cosine similarity embeddings για να αξιολογήσουμε αν το input έχει αλλάξει σημαντικά χωρίς να αλλάξει αντίστοιχα το output 
# — σε αυτή την περίπτωση ζητάει επανεξέταση

def needs_reask(original_input, modified_input, original_output, modified_output, threshold_input=0.7, threshold_output=0.9):
    """
    Επιστρέφει True αν το input άλλαξε αρκετά (input similarity < threshold_input),
    αλλά το output παραμένει σχεδόν ίδιο (output similarity > threshold_output).
    """
    input_sim = util.cos_sim(embedder.encode(original_input, convert_to_tensor=True),
                             embedder.encode(modified_input, convert_to_tensor=True)).item() 

    output_sim = util.cos_sim(embedder.encode(original_output, convert_to_tensor=True),
                              embedder.encode(modified_output, convert_to_tensor=True)).item()

    return input_sim < threshold_input and output_sim > threshold_output

#util.cos_sim , υπολογίζει την cosine similarity μεταξύ δύο embeddings,
#επιστρέφοντας έναν αριθμό μεταξύ -1 και 1 που αντιπροσωπεύει πόσο παρόμοια είναι δύο κείμενα στο νοηματικό επίπεδο




#Run
#τρέχει η ανάλυση counterfactual, συγκρίνει αποτελέσματα,επαναφέρει prompts (αν χρειαστεί)
def run_experiment(sample):
    prompt = build_prompt(sample)
    output = call_llm(prompt)
    print("\n*** FULL RESPONSE ***\n", output, "\n**********************\n")
    finals     = extract_final_advices(output)
    reasonings = extract_reasoning_blocks(output)
    comps      = compare_advices(sample["output"], finals)

    records = []
    for i, c in enumerate(comps):
        mod_input_match = re.search(rf'Modified Input .*?:\s*(.*?)\s*Reasoning:', output, re.DOTALL)
        mod_input = mod_input_match.group(1).strip() if mod_input_match else ""

        if needs_reask(sample["input"], mod_input, sample["output"], finals[i]):
            print(f"\nCounterfactual {i+1} has similar output despite a major change in input. Neads_Reask...\n")
            retry_prompt = f"""
Your previous advice remained almost unchanged despite meaningful changes in the patient input.

Original Input:
\"\"\"{sample['input']}\"\"\"

Modified Input:
\"\"\"{mod_input}\"\"\"

Original Advice:
\"\"\"{sample['output']}\"\"\"

Please now carefully reconsider the Final Advice and explain how the change in input affects your conclusion.

Respond with:

Reasoning: ...
Final Advice: ...
""".strip()
            retry_output = call_llm(retry_prompt)
            retry_reasoning = re.search(r"Reasoning:\s*(.*?)\s*Final Advice:", retry_output, re.DOTALL)
            retry_advice    = re.search(r"Final Advice:\s*(.*)", retry_output)

            finals[i]    = retry_advice.group(1).strip() if retry_advice else finals[i]
            reasonings[i]= retry_reasoning.group(1).strip() if retry_reasoning else reasonings[i]

        records.append({
            "counterfactual_id": i+1,
            "new_advice":  finals[i],
            "reasoning":   reasonings[i] if i < len(reasonings) else "",
            "changed":     c["changed"],
            "opposite":    c["opposite"]
        })

    return records








#Interactive 
#δοκιμές για δικά μας modified inputs σε interactive loop

def interactive_test_loop(sample):
    orig_in = sample["input"]
    orig_ad = sample["output"]
    print("\n=== INTERACTIVE MODE ===")
    print("Original question:", orig_in)
    print("Original advice:",  orig_ad)
    while True:
        cmd = input("\nType modified input (or 'exit'): ").strip()
        if cmd=="exit": break
        manual = f"""
You are a medical assistant.

Modified patient question:
\"\"\"{cmd}\"\"\"

Original question:
\"\"\"{orig_in}\"\"\"

Final Advice:"""
        print("\n>> MODEL:", call_llm(manual))










#Main
#δύο samples
for idx, sample in enumerate(cleaned_healthcaremagic[:2]): 
    print(f"\n*** Running sample {idx+1} ***")
    recs = run_experiment(sample)
    df = pd.DataFrame(recs)
    pd.set_option("display.max_colwidth", None)
    print(df[["counterfactual_id","new_advice","reasoning","changed","opposite"]])
    interactive_test_loop(sample)


*** Running sample 1 ***

*** FULL RESPONSE ***
 Influential Words:

1. Spinning
2. Nauseous
3. Normal

Counterfactuals and Responses:

1. Word 1 (Spinning)
Modified Input (Synonym for 'spinning'): "I woke up this morning feeling the whole room is swaying when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is swaying.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the swaying lessen so i am not sure whether its connected or coincidences.. Thank you doc!"
Reasoning: The patient's symptoms of dizziness and nausea may be related to a vestibular disorder, such as benign paroxysmal positional vertigo (BPPV), which is caused by problems with the inner ear. The fact that the 


Type modified input (or 'exit'):  I briefly felt a bit swaying  this morning while sitting down. I walked to the bathroom steadily, and the dizziness faded right after I freshened up. I didn’t feel nauseous, and I didn’t vomit at all. I took a Panadol just in case, but I honestly feel completely fine now. The dizziness seems to have been very minor and went away quickly. Maybe it was just a normal stomach issue or being tired. Is this something to worry about, or is it normal? Thanks in advance!



>> MODEL: Thank you for sharing your concerns with me. Based on the information provided, it's possible that you may be experiencing a temporary dizziness or lightheadedness due to a variety of factors. It could be related to a stomach issue, fatigue, or even a minor inner ear problem. However, without further evaluation and testing, it's difficult to pinpoint the exact cause.

I would recommend that you follow up with your primary care physician to discuss your symptoms in more detail. They may refer you to a specialist such as an ear, nose, and throat (ENT) doctor or a neurologist for further evaluation. In the meantime, it's important to stay hydrated, get plenty of rest, and avoid any strenuous activities that may exacerbate your symptoms.

If you experience any sudden changes in your symptoms or if they persist or worsen over time, please don't hesitate to seek medical attention immediately.

Remember, it's always better to be safe than sorry when it comes to your health. Take ca


Type modified input (or 'exit'):  I felt a slight a momentary swaying sensation earlier this morning when I got up from bed, but it passed almost immediately. I continued my day as usual and haven’t felt anything similar since. There was no nausea or discomfort, and I feel completely fine now. It might have just been from standing up too fast or not sleeping enough. I’m just checking in to make sure it’s nothing to worry about. So, I don’t really need to see a doctor for this, right? Thanks a lot!



>> MODEL: Thank you for sharing your symptoms with me. Based on what you've told me, it seems like you may be experiencing a common condition called benign paroxysmal positional vertigo (BPPV). BPPV is caused by small crystals in the inner ear becoming dislodged and moving into the canals of the inner ear, which can cause a feeling of spinning or movement when you change positions.

It's possible that your symptoms are related to BPPV, but it's also possible that they could be caused by another condition such as a viral infection or a more serious inner ear problem. To determine the cause of your symptoms, I would recommend scheduling an appointment with a specialist such as an otolaryngologist (ENT doctor) or a neurologist. They can perform a thorough examination and order any necessary tests to determine the cause of your symptoms.

In the meantime, there are some things you can try to help manage your symptoms. Avoid standing up too quickly, as this can make your symptoms worse. Tr

KeyboardInterrupt: Interrupted by user